# Create Collection of Predictive Words per Personality Trait using LinearSVC

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import chi2
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

<br><br>
### Import Data

In [3]:
data = pd.read_csv('../../data/cleaned/expanded_mbti_df.csv')

In [4]:
data.head(2)

,type,posts,comp_score,neg_score,neu_score,pos_score,post_count,avg_word_count,posts_cleaned,cleaned_comp_score,...,diff_comp_init-no_punct,E_I,N_S,F_T,J_P,E_I_code,N_S_code,F_T_code,J_P_code,type_code
0,INFJ,"[""'http://www.youtube.com/watch?v=qsXHcwe3krw""...",0.9877,0.054,0.829,0.116,50,90,['enfp and intj moments sportscenter not top t...,0.9839,...,0.0074,I,N,F,J,0,1,1,1,111
1,ENTP,"[""'I'm finding the lack of me in these posts v...",0.9994,0.068,0.752,0.180,50,138,"[""'I'm finding the lack of me in these posts v...",0.9993,...,0.0009,E,N,T,P,1,1,0,0,1100


<br><br>
### Custom Stopword Collection

In [56]:
with open('../../data/full_stopwords.txt', 'r') as filehandle:
    custom_stopwords = [words.rstrip() for words in filehandle.readlines()]

<br><br>
### TFDIF Vectorizer Model: using *stop_words=custom_stopwords*

In [194]:
targets = ['E_I', 'N_S', 'F_T', 'J_P']

In [195]:
targets_dict_tf = {}

for target in targets:   
    target_dict_L1 = {}
    target_dict_L2 = {}
    ngram_pos_dict = {}
    ngram_neg_dict = {}

    for i in range(1,5):
        word_pos_dict = {}
        word_neg_dict = {}
        
        print('FLAG: '+target+'...'+str(i))
        classifier = LinearSVC(C=1.0, class_weight="balanced", max_iter = 10_000)
        tf_idf = Pipeline([
                    ('tfidf', TfidfVectorizer(stop_words=custom_stopwords, ngram_range=(i,i), min_df=10)),
                    ("classifier", classifier)])
        tf_idf.fit(data.posts_no_digits, data[target])
        coefs = tf_idf.named_steps["classifier"].coef_
        coefs.tolist()
        feature_names = tf_idf.named_steps["tfidf"].get_feature_names()
        coefs_and_features = list(zip(coefs[0], feature_names))
        positive_results = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)[:30]
        negative_results = sorted(coefs_and_features, key=lambda x: x[0])[:30]

        for result in positive_results:
            word_pos_dict['word_'+str(len(word_pos_dict)+1)] = {'coef':abs(result[0]), 'word':result[1]}
        ngram_pos_dict['ngram_'+str(i)] = word_pos_dict
        target_dict_L1.update(ngram_pos_dict)

        for result in negative_results:
            word_neg_dict['word_'+str(len(word_neg_dict)+1)] = {'coef':abs(result[0]), 'word':result[1]}
        ngram_neg_dict['ngram_'+str(i)] = word_neg_dict
        target_dict_L2.update(ngram_neg_dict)
    
    targets_dict_tf[target[0]] = target_dict_L1
    targets_dict_tf[target[2]] = target_dict_L2

FLAG: E_I...1
FLAG: E_I...2
FLAG: E_I...3
FLAG: E_I...4
FLAG: N_S...1
FLAG: N_S...2
FLAG: N_S...3
FLAG: N_S...4
FLAG: F_T...1
FLAG: F_T...2
FLAG: F_T...3
FLAG: F_T...4
FLAG: J_P...1
FLAG: J_P...2
FLAG: J_P...3
FLAG: J_P...4


In [225]:
reindexed_target_dict = {}

for targetKey, ngramDict in targets_dict_tf.items():
    for ngramKey, wordDict in ngramDict.items():
        for word_indexKey, word_indexValue in wordDict.items():
            reindexed_target_dict[(targetKey, ngramKey, word_indexKey)] = word_indexValue

ngrams_tf_df = pd.DataFrame(reindexed_target_dict).T
ngrams_tf_df

coef                                      word
E ngram_1 word_1    1.99503                                    nature
          word_2    1.95812                                    family
          word_3    1.84229                                     quiet
          word_4    1.83612                                  inferior
          word_5    1.75728                                      mind
...                     ...                                       ...
P ngram_4 word_26  0.669775               acts service physical touch
          word_27   0.66174                flickr explore look random
          word_28  0.638434  hierarchies methods introverted thinking
          word_29  0.636039     life circumstances useful information
          word_30  0.627231                   welcome hope enjoy time

[960 rows x 2 columns]

In [196]:
targets_dict_cv = {}

for target in targets:   
    target_dict_L1 = {}
    target_dict_L2 = {}
    ngram_pos_dict = {}
    ngram_neg_dict = {}

    for i in range(1,5):
        word_pos_dict = {}
        word_neg_dict = {}
        
        print('FLAG: '+target+'...'+str(i))
        classifier = LinearSVC(C=1.0, class_weight="balanced", max_iter = 10_000)
        cvect = Pipeline([
                    ('countvect', CountVectorizer(stop_words=custom_stopwords, ngram_range=(i,i), min_df=10)),
                    ("classifier", classifier)])
        cvect.fit(data.posts_no_digits, data[target])
        coefs = cvect.named_steps["classifier"].coef_
        coefs.tolist()
        feature_names = cvect.named_steps["countvect"].get_feature_names()
        coefs_and_features = list(zip(coefs[0], feature_names))
        positive_results = sorted(coefs_and_features, key=lambda x: x[0], reverse=True)[:30]
        negative_results = sorted(coefs_and_features, key=lambda x: x[0])[:30]

        for result in positive_results:
            word_pos_dict['word_'+str(len(word_pos_dict)+1)] = {'coef':abs(result[0]), 'word':result[1]}
        ngram_pos_dict['ngram_'+str(i)] = word_pos_dict
        target_dict_L1.update(ngram_pos_dict)

        for result in negative_results:
            word_neg_dict['word_'+str(len(word_neg_dict)+1)] = {'coef':abs(result[0]), 'word':result[1]}
        ngram_neg_dict['ngram_'+str(i)] = word_neg_dict
        target_dict_L2.update(ngram_neg_dict)
    
    targets_dict_cv[target[0]] = target_dict_L1
    targets_dict_cv[target[2]] = target_dict_L2

FLAG: E_I...1
FLAG: E_I...2
FLAG: E_I...3


/Users/rebeccawright/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


FLAG: E_I...4


/Users/rebeccawright/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


FLAG: N_S...1
FLAG: N_S...2
FLAG: N_S...3


...

/Users/rebeccawright/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


FLAG: J_P...1
FLAG: J_P...2
FLAG: J_P...3


/Users/rebeccawright/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


FLAG: J_P...4


/Users/rebeccawright/opt/anaconda3/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [222]:
reindexed_target_dict = {}

for targetKey, ngramDict in targets_dict_cv.items():
    for ngramKey, wordDict in ngramDict.items():
        for word_indexKey, word_indexValue in wordDict.items():
            reindexed_target_dict[(targetKey, ngramKey, word_indexKey)] = word_indexValue

ngrams_cv_df = pd.DataFrame(reindexed_target_dict).T
ngrams_cv_df

coef                                     word
E ngram_1 word_1   0.246644                                 organize
          word_2   0.219742                                  letting
          word_3   0.212582                                    shoot
          word_4   0.194911                                 climbing
          word_5   0.191951                               statistics
...                     ...                                      ...
P ngram_4 word_26  0.602789               tapatalk sent iphone using
          word_27  0.592866    life circumstances useful information
          word_28  0.582466                like tea specifically hot
          word_29  0.582466                 tea specifically hot tea
          word_30  0.556023  personality profile closest alternative

[960 rows x 2 columns]

In [223]:
ngrams_cv_df.to_csv('ngrams_cv_svc.csv')

In [226]:
ngrams_tf_df.to_csv('ngrams_tf_svc.csv')